## Local tests

# prerequisites:

In [1]:
import boto3
import sagemaker

region = boto3.Session().region_name
# TODO: insert your role here
role = ''
default_bucket = sagemaker.session.Session().default_bucket()
training_pipeline_name = "TrainingAbalonePipeline"
source_data_bucket = "mlops-sagemaker-project"

## upsert pipeline

In [2]:
# TODO: insert Abalone project root
ABALONE_PROJECT_ROOT =''

In [3]:
import sys
sys.path.append(f'{ABALONE_PROJECT_ROOT}/pipelines')

In [4]:
from mlops_utilities.actions import upsert_pipeline

In [ ]:
upsert_pipeline(pipeline_module='src', pipeline_package='training_pipeline',
                pipeline_name='mlops-utilities-upsert', config_type='training_pipeline.defaults', role=role)

## run pipeline

In [6]:
from mlops_utilities.actions import run_pipeline

In [ ]:
run_pipeline(
    pipeline_name=training_pipeline_name,
    execution_name_prefix='latest',
    dryrun=False,
    pipeline_params={
        'RegisterModel': False,
        "InputDataS3Uri": f"s3://{source_data_bucket}/abalonedata/data.csv",
        "InputLabelS3Uri": f"s3://{source_data_bucket}/abalonelabels/data.csv",
    }
)

## create endpoint

In [8]:
boto_sess = boto3.Session()
sm = boto_sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=boto_sess)
endpoint_name = 'TestEndpoint'

instance_type = 'ml.m5.large'
instance_count = 1

1. create model

In [9]:
aws_region = 'us-east-1'
sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

In [10]:
import datetime
import os
from typing import Any, Dict, List

from mlops_utilities import helpers


def _get_sagemaker_project_tags() -> List[Dict[str, str]]:
    tags = []

    project_id = os.getenv('SAGEMAKER_PROJECT_ID')
    if project_id is not None:
        tags.append({'Key': 'sagemaker:project-id', 'Value': project_id})

    project_name = os.getenv('SAGEMAKER_PROJECT_NAME')
    if project_name is not None:
        tags.append({'Key': 'sagemaker:project-name', 'Value': project_name})

    return tags


current_datetime_utc_str = helpers.get_datetime_str(datetime.datetime.now(datetime.timezone.utc))
sagemaker_project_tags = _get_sagemaker_project_tags()

In [ ]:
model_package_arn = 'arn:aws:sagemaker:us-east-1:311638508164:model-package/test-sokolov/1'
model_arn = helpers.create_model_from_model_package(
    sagemaker_client=sagemaker_client,
    model_name='model',
    model_package_arn=model_package_arn,
    execution_role=role,
    tags=sagemaker_project_tags,
)
print(f'Model {model_arn} has been created successfully', model_arn)

2. create endpoint config

In [12]:
s3_data_capture_output_prefix = None
endpoint_config_name = f'{endpoint_name}-config-{current_datetime_utc_str}'
data_capture_config = helpers._generate_data_capture_config(
    s3_data_capture_output_prefix) if s3_data_capture_output_prefix is not None else None

In [ ]:
from mlops_utilities.actions import create_endpoint

boto_sess = boto3.Session()
sm = boto_sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=boto_sess)

create_endpoint(model_package_arn, sagemaker_session,
                instance_count, instance_type, endpoint_name,
                data_capture_config, role=role)

## update endpoint

In [14]:
from mlops_utilities.actions import update_endpoint

In [15]:
evaluate_s3 = 's3://sagemaker-mlops-p-sq33brxtdjtx/EvaluateAbaloneModel-f4800dc7aed96399c5afac406ea9afe0/output/evaluation/evaluation.json'
metric_path = 'regression_metrics/mse/value'

In [ ]:
boto_sess = boto3.Session()
sm = boto_sess.client("sagemaker")
sm.describe_endpoint_config(EndpointConfigName=endpoint_name)

In [ ]:
update_endpoint(sagemaker_client=sm, endpoint_name=endpoint_name, data_capture_config=data_capture_config,
                model_statistics_s3_uri=evaluate_s3, metric=metric_path)

## deploy model

In [19]:
from mlops_utilities.actions import deploy_model

In [20]:
boto_sess = boto3.Session()
sm = boto_sess.client("sagemaker")
model_package_group_name = 'test-Sokolov'
data_capture_s3_uri = 's3://kris-mlops-utilities-test/'

In [ ]:
sm.list_model_packages(
    ModelApprovalStatus='Approved',
    ModelPackageGroupName=model_package_group_name,
    SortBy='CreationTime',
    SortOrder='Descending',
    MaxResults=1,
)

In [ ]:
deploy_model(sagemaker_session, model_package_group_name, instance_type, instance_count, endpoint_name, data_capture_s3_uri, role=role)